In [5]:
## Data infestion
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
text_documents = loader.load()


'Document'

In [88]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [12]:
## Data ingestion using web based loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_path =("https://www.bbc.com/sport/football/articles/cyrrjv3e4zro"),
    bs_kwargs=dict(parse_only = bs4.SoupStrainer(
        class_=("zephr_join_beta","ssrcss-181c4hk-SectionWrapper eustbbg0",
        "ssrcss-4rxmy3-PageStack e1mcntqj2","ssrcss-l6cntj-ContentStack e1mcntqj0",
        "ssrcss-irv5dn-Zone e1mcntqj4","ssrcss-irv5dn-Zone e1mcntqj4",
        "ssrcss-1ocoo3l-Wrap e42f8511","ssrcss-1y7k614-FooterStack e1mcntqj1")
        )
    )
)

web_load = loader.load()
print(web_load)

[Document(page_content='BBC HomepageSkip to contentAccessibility HelpYour accountHomeNewsSportBusinessInnovationCultureTravelMore menuMore menuSearch BBCHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveClose menuBBC SportMenuHomeFootballCricketFormula 1Rugby UTennisGolfAthleticsCyclingMoreA-Z SportsAmerican FootballAthleticsBasketballBoxingCricketCyclingDartsDisability SportFootballFormula 1Gaelic GamesGolfGymnasticsHorse RacingMixed Martial ArtsMotorsportNetballOlympic SportsRugby LeagueRugby UnionSnookerSwimmingTennisWinter SportsFull Sports A-ZMore from SportEnglandScotlandWalesNorthern IrelandNews FeedsHelp & FAQsFootballScores & FixturesTablesUEFA Euro 2024GossipTransfersTop ScorersWomenEuropeanAll TeamsLeagues & CupsMessi farewell? Vinicius Jr for Ballon d\'Or? Copa America set to startImage source, Getty ImagesImage caption, Lionel Messi is third top scorer in international men\'s football ever with 106 goalsEmlyn BegleyBBC Sport journalistPublished19 June 2024Updated 2

In [70]:
## using pdfs

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("jose's_plan.pdf")
docs = loader.load()
docs

[Document(page_content='José\'s Plan\nAmalendu P\nTottenham Hotspur hasn\'t won a trophy in the last 13 years, since \nthey won the Carling cup in 2008 beating Chelsea at Wembley. \nFor Spurs Chairman Daniel Levy, the most logical move was to \nbring in a special manager. Someone who has plenty of experience\nwinning trophies and making history. \nJosé Mourinho was appointed Manager and Head Coach of \nTottenham Hotspur on 20th November 2019, following the \nsacking of ex-coach Mauricio Pochettinho. In his press conference\na week later, he was asked, "Do you think, not winning the final in\nThe Champions League pay a toll in the squad of Mauricio \nPochettinho ?". José replied, "I don\'t know, because I\'ve never lost\na champions league final." This shows José\'s mentality and that he\nwas under no pressure. Spurs were 14th when he was appointed. \nThey ended up finishing 6th in the 2019-2020 season, nabbing a \nEuropa League spot from Wolverhampton Wanderers only on goal\ndifference

In [109]:
## Dividing the text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
final_docs = text_splitter.split_documents(docs)

In [110]:
## Vector embeddings and Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



In [107]:
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r1

[[-0.013434873893857002,
  -0.03690733015537262,
  -0.02270950935781002,
  0.02707621455192566,
  -0.009050454944372177,
  -0.006741381250321865,
  -0.0031774018425494432,
  0.001790613285265863,
  -0.0029444184619933367,
  -0.009304696694016457,
  0.05261484906077385,
  -0.05360826477408409,
  0.03472718968987465,
  0.061702292412519455,
  -0.008199655450880527,
  -0.007371014449745417,
  -0.007594195660203695,
  0.0021142028272151947,
  0.03303099796175957,
  -0.0045167068019509315,
  -0.06418141722679138,
  0.0034098229371011257,
  -0.01186743937432766,
  0.0009274149197153747,
  0.07993786782026291,
  -0.007561027072370052,
  0.013167111203074455,
  -0.025330791249871254,
  -0.017058806493878365,
  -0.010063146241009235,
  -0.0484318733215332,
  0.003608418395742774,
  0.00022257772798184305,
  0.022612858563661575,
  1.4194664572642068e-06,
  -0.020168473944067955,
  0.01795148476958275,
  -0.00437674717977643,
  -0.05995224043726921,
  -0.026209505274891853,
  0.04921156913042068

In [108]:


from langchain_community.vectorstores import Chroma

# embeddding_model = ollama_emb
# try:
#     # Create a Chroma collection with the correct dimensionality
#     db = Chroma.from_documents(final_docs, embedding_model)
# except InvalidDimensionException as e:
#     print(f"Error: {e}")
#     # Handle the error: This can involve creating a new collection with the correct dimension
#     # Assuming we need to create a new collection:
#     db = Chroma.from_documents(final_docs, embedding_model, collection_name="new_collection_name")

db = Chroma.from_documents(final_docs,ollama_emb)

InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 4096

In [75]:
query = "Who is the chairman of Tottenham Hostpur ?"
result = db.similarity_search(query)

In [76]:
result

[Document(page_content="Vinicius is the only rotation striker who could adapt perfectly to \nJosé's choice of playstyle. If either Kane or Son gets injured, it \nseems that the season would be over for Spurs.\nThey currently stand 3rd in the table and was 1st before \nChristmas, before clumsily dropping points against Liverpool and \nCrystal Palace. The title race, I would say, is still on as there has \nbeen no run-away leader. Spurs have earned their spot in the \nCarabao Cup Final, which could earn them their first competetive \ntrophy in 13 years.\nFinal Verdict :\nThis could be Tottenham Hotspurs' breakthrough season. If they \ncan avoid injuries and hold onto leads efficiently, I believe it'll all \ngo according to José's Plan.", metadata={'page': 3, 'source': "jose's_plan.pdf"}),
 Document(page_content='efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, gre

# Creating LLM Chain

In [36]:
# import and ready the llm, here, we use groq api
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
import dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [84]:
prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only the context provided.
    Think step by step and provide a detailed answer.
    Just give the final precise answer, no need to explain.
    <context>
    {context}
    </context>
    Question : {input}
    """
    )


In [85]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000023EDD4834C0>)

In [86]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)
retrieval_chain.invoke({"input":"What is Tottenham's conventional playstyle ?",
                        "context":result})['answer']

'Counter attack.'